In [2]:
%%capture
!pip install wikidataintegrator

In [36]:
from rdflib import Graph, URIRef
from wikidataintegrator import wdi_core, wdi_login
from datetime import datetime
import copy
import pandas as pd
import getpass

In [37]:
print("username:")
username = input()
print("password:")
password = getpass.getpass()
login = wdi_login.WDLogin(user=username, pwd=password)

username:
andrawaag
password:
········


In [32]:
# functions
def createOBOReference(doid):
    statedin = wdi_core.WDItemID(obowditem, prop_nr="P248", is_reference=True)
    retrieved = datetime.now()
    timeStringNow = retrieved.strftime("+%Y-%m-%dT00:00:00Z")
    refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True)
    id = wdi_core.WDExternalID(oboid, prop_nr=oboidwdprop, is_reference=True)
    return [statedin, refRetrieved, id]

In [34]:
oboid = "GO:0036310"
obouri = "http://purl.obolibrary.org/obo/"+oboid.replace(":", "_")
obouri

# wikidata
obowditem = "Q135085" #Gene Ontology
oboidwdprop = "P686" #gene ontology id
oboontology = "GO"

## Fetch the OBO ontology
obog = Graph()
obog.parse(f"http://www.ontobee.org/ontology/rdf/{oboontology}?iri="+obouri, format="xml")

## Fetch Wikidata part of the OBO ontology
query = f"""
SELECT * WHERE {{?item wdt:{oboidwdprop} '{oboid}'}}
"""
qid = wdi_core.WDFunctionsEngine.execute_sparql_query(query, as_dataframe=True)
if len(qid) >0:
    qid = qid.iloc[0]["item"].replace("http://www.wikidata.org/entity/", "")
else:
    qid = None

# Bot
## ShEx precheck

if qid:
    item = wdi_core.WDItemEngine(wd_item_id=qid) 
    # precheck = item.check_entity_schema(eid="E323", output="result")
    #if not precheck["result"]:
    #    print(qid + " needs fixing to conform to E323")
    #    quit()
print("continue")

obo_reference = createOBOReference(oboid)

# Statements build up
## OBO ontology generic
statements = []
# OBO ID 
statements.append(wdi_core.WDString(value=oboid, prop_nr=oboidwdprop, references=[copy.deepcopy(obo_reference)]))
# exact match (P2888)
statements.append(wdi_core.WDUrl(value=obouri, prop_nr="P2888", references=[copy.deepcopy(obo_reference)]))

## OBO resource specific 
### Gene Ontology
gotypes =  {"biological_process": "Q2996394", 
 "molecular_function": "Q14860489", 
 "cellular_component": "Q5058355",
}

for gotype in obog.objects(predicate=URIRef("http://www.geneontology.org/formats/oboInOwl#hasOBONamespace")):
    statements.append(wdi_core.WDItemID(gotypes[str(gotype)], prop_nr="P31", references=[copy.deepcopy(obo_reference)]))

#external identifiers based on skos:exactMatch
for extID in obog.objects(predicate=URIRef("http://www.w3.org/2004/02/skos/core#exactMatch")):
    # if "MESH:" in extID:
    #    statements.append(wdi_core.WDExternalID(row["exactMatch"].replace("MESH:", ""), prop_nr="P486", references=[copy.deepcopy(do_reference)]))
    if "NCI:" in extID:
        statements.append(wdi_core.WDExternalID(row["exactMatch"], prop_nr="P1748", references=[copy.deepcopy(do_reference)]))
    if "ICD10CM:" in extID:
        statements.append(wdi_core.WDExternalID(row["exactMatch"], prop_nr="P4229", references=[copy.deepcopy(do_reference)]))
    if "UMLS_CUI:" in extID:
        statements.append(wdi_core.WDExternalID(row["exactMatch"], prop_nr="P2892", references=[copy.deepcopy(do_reference)]))
item = wdi_core.WDItemEngine(data=statements, keep_good_ref_statements=True)
print(item.write(login))


continue
Q21111094
